# Lab 3 - The Matrix Form and The Duality Theory

<b>Information on group members:</b><br>
1) 150284, Sofya Aksenyuk <br>
2) 150281, Uladzimir Ivashka

In [2]:
from pulp import *  
import numpy as np
import pandas as pd
import itertools
from IPython.display import display

# 1) The Matrix Form - Fundamental Insight (finish this part to get 3.0)

1.1) Given is the below (primal) linear problem:

primal problem:<br>
MAXIMIZE<br>
4*x1 + 2*x2 + 3*x3 <br>

SUBJECT TO<br>
1_constraint: x1 + x2 <= 10<br>
2_constraint: 2*x2 + x3 <= 12<br>
3_constraint: 3*x1 + 2*x3 <= 15<br>
4_constraint: x1 + x2 + x3 <= 20<br>

VARIABLES<br>
x1 Continuous<br>
x2 Continuous<br>
x3 Continuous<br>

1.2) Use the PuLP library to solve the above problem. Identify the optimal solution: the values for basic variables and the corresponding value for the objective function.

In [3]:
### Model here:
### TODO
model = LpProblem(name="problem", sense=LpMaximize)

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)
x3 = LpVariable(name="x3", lowBound=0)

model += (x1 + x2 <= 10, "#1 constraint")
model += (2 * x2 + x3 <= 12, "#2 constraint")
model += (3 * x1 + 2 * x3 <= 15, "#3 constraint")
model += (x1 + x2 + x3 <= 20, "#4 constraint")

obj_func = 4 * x1 + 2 * x2 + 3 * x3
model += obj_func

model

problem:
MAXIMIZE
4*x1 + 2*x2 + 3*x3 + 0
SUBJECT TO
#1_constraint: x1 + x2 <= 10

#2_constraint: 2 x2 + x3 <= 12

#3_constraint: 3 x1 + 2 x3 <= 15

#4_constraint: x1 + x2 + x3 <= 20

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous

In [4]:
### Solve here
### TODO
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}\n")

print(f"objective: {model.objective.value()}\n")

for name, constraint in model.constraints.items():  print(f"{name}: {constraint.value()}")

print()    
    
for var in model.variables():
    print(f"{var}:", var.value())

status: 1, Optimal

objective: 31.428571379999998

#1_constraint: 0.0
#2_constraint: 5.999999985739635e-08
#3_constraint: -7.999999906971311e-08
#4_constraint: -9.14285714

x1: 4.4285714
x2: 5.5714286
x3: 0.85714286


1.3) In this exercise, you are asked to identify ALL basic (feasible and not) solutions to the above problem. We will do this naively. Specifically, you are asked to use the fundamental insight to build a final simplex tableau for each possible base. Therefore, you need first to initialize the data: c, b, A matrixes, and it is suggested to initialize the auxiliary matrix M defined as M = A + (concatenate) I (identity matrix). Note that the problem should be formulated in the augmented form. Then, you have to iterate over each possible base B, compute B-1, and other relevant parts for the simplex tableau.<br><br>
a) Identify the optimal solution using the optimality condition; print it (Z value and values for basic variables); compare thus derived solution with the optimum found using the PuLP library (obviously, both solutions should be the same). <br>
b) Count the number of feasible and infeasible solutions. How many (all) basic solutions to the problem can be identified? <br><br>
It is suggested to use the NumPy library for performing matrix operations. 

Max Z = 4x1 + 2x2 + 3x3

AUGMENTED FORM:

(1) Z - 4x1 - 2x2 - 3x3 = 0

(2) x1 + x2 + x4 = 10

(3) 2x2 + x3 + x5 = 12

(4) 3x1 + 2x3 + x6 = 15

(5) x1 + x2 + x3 + x7 = 20

s.t. x1..x7 >= 0 

In [5]:
### Initialize the data here:
### TODO
c = np.array([4, 2, 3])
b = np.array([10, 12, 15, 20]).reshape(4, 1)
A = np.array([
    [1, 1, 0], 
    [0, 2, 1],
    [3, 0, 2],
    [1, 1, 1]
])
M = np.array(np.concatenate((A, np.identity(A.shape[0])), axis=1))
base = np.array([3, 4, 5, 6])

<b> Important note: the below is just a proposition. You can solve the problem in your own way. </b>

You can define an auxiliary method constructing a final simplex tableau for a given base.  Here, "base" is a list of columns (integers) for the base. Note that the functions in python can return multiple objects and you can use this functionality to return<br>
- the inversed base<br>
- coefficients in the row 0 for slack variables<br>
- right side values (except the objective function value)<br>
- the objective function value<br>
- the coefficients for decision variables in row 0 <br>
- the coefficients for decision variables in rows 1+<br>

Note that if BI cannot be built (it is possible), the method may return None in order to notify the executive method about this exception. 



In [6]:
def getFinalTableau(base, c, b, A, M):
    I = np.identity(len(base))
    aug_func = np.concatenate((base.copy(), list(.0 for x in range(len(base)))), axis=0)
    x_b = base.copy()
    c_b = list(0 for x in range(len(base)))
    B = M[:, base]
    B_inv = np.linalg.inv(B)
    dec = c.copy()

    row_0 = np.concatenate((-c, [0] * len(base), [0]), axis=0) + c_b @ B_inv @ np.concatenate((A, I, b), axis=1)
    row_1 = B_inv @ np.concatenate((A, I, b), axis=1)
    
    primal_solutions = []
    for_dual = []
    z = []
    
    while min(row_0) < 0:
        enter_idx = np.where(dec == max(dec))[0][0]
        dec = np.delete(dec, enter_idx)
        MRT = np.divide(b.T, A[:, enter_idx])[0]
        leave_idx = np.where(MRT == min(MRT))[0][0]
        x_b[leave_idx] = enter_idx
        c_b[leave_idx] = -row_0[enter_idx]
        B[:, leave_idx] = A[:, enter_idx]
        B_inv = np.linalg.inv(B)
 
        row_0 = np.concatenate((-c, [0] * len(base), [0]), axis=0) + c_b @ B_inv @ np.concatenate((A, I, b), axis=1)
        row_1 = B_inv @ np.concatenate((A, I, b), axis=1)
        
        primal_solutions.append(list(row_0)[:-1])
        z.append(list(row_0)[-1])
        
    return np.around(np.concatenate(([row_0], row_1), axis=0), 2), x_b, primal_solutions, z

In [7]:
df, new_base, primal_solutions, z = getFinalTableau(base, c, b, A, M)
df = pd.DataFrame(df, columns=["x1", "x2", "x3", "x4", "x5", "x6", "x7", "RS"])
df.insert(0, "Z", [1, 0, 0, 0, 0])
df.insert(0, "Base", ["Z"] + [f"x{i + 1}" for i in new_base])
df

<ipython-input-6-1715238db2f6>:20: RuntimeWarning: divide by zero encountered in divide
  MRT = np.divide(b.T, A[:, enter_idx])[0]


,Base,Z,x1,x2,x3,x4,x5,x6,x7,RS
0,Z,1,0.0,0.0,0.67,0.0,1.0,1.33,0.0,32.0
1,x4,0,0.0,0.0,-1.17,1.0,-0.5,-0.33,0.0,-1.0
2,x2,0,0.0,1.0,0.50,0.0,0.5,0.00,0.0,6.0
3,x1,0,1.0,0.0,0.67,0.0,0.0,0.33,0.0,5.0
4,x7,0,0.0,0.0,-0.17,0.0,-0.5,-0.33,1.0,9.0


## 2) The Duality Theory (finish this part + part 1 + to get 5.0)

2.1) Model the dual problem to the above solved primal one, using the PuLP library. Then, solve it and compare the derived optimum with the optimum for the primal problem. Are they equal? Answer: Yes.

Dual problem:<br>

Min W = 10y1 + 12y2 + 15y3 + 20y4<br>

SUBJECT TO<br>
1_constraint: y1 + 3y3 + y4 >= 4<br>
2_constraint: y1 + 2y2 + y4 >= 2<br>
3_constraint: y2 + 2y3 + y4 >= 3<br>

VARIABLES<br>
y1 Continuous<br>
y2 Continuous<br>
y3 Continuous<br>
y4 Continuous<br>

Solution: (0 1 1.34 0 | 0 0 0.67)

W' = Z ~= 32

In [8]:
### Model here:
### TODO
dual_model = LpProblem(name="dual_problem", sense=LpMinimize)

y1 = LpVariable(name="y1", lowBound=0)
y2 = LpVariable(name="y2", lowBound=0)
y3 = LpVariable(name="y3", lowBound=0)
y4 = LpVariable(name="y4", lowBound=0)

dual_model += (y1 + 3 * y3 + y4 >= 4, "#1 constraint")
dual_model += (y1 + 2 * y2 + y4 >= 2, "#2 constraint")
dual_model += (y2 + 2 * y3 + y4 >= 3, "#3 constraint")

dual_obj_func = 10 * y1 + 12 * y2 + 15 * y3 + 20 * y4
dual_model += dual_obj_func

dual_model

dual_problem:
MINIMIZE
10*y1 + 12*y2 + 15*y3 + 20*y4 + 0
SUBJECT TO
#1_constraint: y1 + 3 y3 + y4 >= 4

#2_constraint: y1 + 2 y2 + y4 >= 2

#3_constraint: y2 + 2 y3 + y4 >= 3

VARIABLES
y1 Continuous
y2 Continuous
y3 Continuous
y4 Continuous

In [9]:
### Solve here
### TODO
status_dual = dual_model.solve()
print(f"status: {dual_model.status}, {LpStatus[dual_model.status]}\n")

print(f"objective: {dual_model.objective.value()}\n")

for name, constraint in dual_model.constraints.items():  print(f"{name}: {constraint.value()}")

print()    
    
for var in dual_model.variables():
    print(f"{var}:", var.value())

status: 1, Optimal

objective: 31.42857072

#1_constraint: -1.2999999965401798e-07
#2_constraint: -9.99999993922529e-09
#3_constraint: -8.999999989711682e-08

y1: 0.57142857
y2: 0.71428571
y3: 1.1428571
y4: 0.0


Augumented form:

(1) -W - 10y1 - 12y2 - 15y3 -20y4 + M y6_hat + M y8_hat + M y10_hat = 0

(2) y1 + 3y3 + y4 - y5 + y6_hat = 4

(3) y1 + 2y2 + y4 - y7 + y8_hat = 2

(4) y2 + 2y3 + y4 - y9 + y10_hat = 3

In [10]:
def get_dual(primal_solution, break_point):
    dual_solutions = []

    for i in primal_solution:
        dual_solutions.append(i[break_point:] + i[:break_point])
    
    return dual_solutions

dual_solutions = get_dual(primal_solutions, len(new_base))

2.2) This exercise is based on the exercise 1.3 (copy & paste solution). Here, you are asked to iterate over all basic solutions (as in 1.3) and store them along with their complementary dual solutions. Solutions should be stored in the PRIMAL_DUAL_SOLUTIONS list and sorted according to the objective value Z = W. Analyze their optimality and feasibility. Finally, you have to display all basic solutions along with their complementary solutions (you can use the provided piece of code written using the pandas library). <br><br>

PRIMAL_DUAL_SOLUTIONS is defined as a table consisting of n rows, where n is the number of basic solutions to the problem, and 21 columns. The columns are defined as follows:<br>
Col. 1: The objective value Z<br>
Col. 2-4: The values for decision variables (primal solution)<br>
Col. 5-8: The values for slack variables (primal solution)<br>
Col. 9: P_F = Y or N, Y/N = primal solution is feasible/infeasible<br>
Col. 10: P_O = Y or N, Y/N = primal solution is optimal/is not optimal<br>
Col. 11: P_STATE = -/suboptimal/superoptimal/optimal; depends on P_F and P_O (primal)<br>
Col. 12: D_STATE = -/suboptimal/superoptimal/optimal; depends on D_F and D_O (dual)<br>
Col. 13: D_F = Y or N, Y/N = dual solution is feasible/infeasible<br>
Col. 14: D_O = Y or N, Y/N = dual solution is optimal/is not optimal<br>
Col. 15-18: The values for decision variables (dual solution)<br>
Col. 19-21: The values for surplus variables (dual solution)<br><br>

Reminder: sort solutions according to Z; analyze how their states change with the increase of Z.

In [11]:
### DISPLAY STORED PAIRS OF SOLUTIONS
PF = np.all(np.array(primal_solutions) >= 0, axis=1)
PO = np.all(np.array(dual_solutions) >= 0, axis=1)
DO, DF = PF[:], PO[:]
conditions = (PF == True) & (PO == False), (PF == False) & (PO == False), (PF == True) & (PO == True), (PF == False) & (PO == True)
res = ["Sub", "-", "Opt", "Super"]
p_state = np.select(conditions, res)
res2 = ["Super", "-", "Opt", "Sub"]
d_state = np.select(conditions, res2)

df = pd.DataFrame(z)

df[["x1", "x2", "x3", "slack1", "slack2", "slack3", "slack4"]] = primal_solutions

df['P_F'] = PF
df['P_O'] = PO
df['P_STATE'] = p_state
df['D_STATE'] = d_state
df['D_F'] = DF
df['D_O'] = DO

df[["y1", "y2", "y3", "y4", "sur1", "sur2", "sur3"]] = dual_solutions

display(df.style.set_properties(**{
       'width': '230px',
       'max-width': '230px',
   }))

,0,x1,x2,x3,slack1,slack2,slack3,slack4,P_F,P_O,P_STATE,D_STATE,D_F,D_O,y1,y2,y3,y4,sur1,sur2,sur3
0,20.000000,0.000000,-2.000000,-0.333333,0.000000,0.000000,1.333333,0.000000,False,False,-,-,False,False,0.000000,1.333333,0.000000,0.000000,-2.000000,-0.333333,0.000000
1,32.000000,0.000000,0.000000,0.666667,0.000000,1.000000,1.333333,0.000000,True,True,Opt,Opt,True,True,1.000000,1.333333,0.000000,0.000000,0.000000,0.666667,0.000000
